In [3]:
!pip install faker
!pip install citipy
!jupyter nbextension enable --py --sys-prefix widgetsnbextension
!pip install gmaps
!jupyter nbextension enable --py --sys-prefix gmaps

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


Enabling notebook extension jupyter-gmaps/extension...
      - Validating: OK


In [18]:

import pandas as pd
import requests
import os
import sys
sys.path.insert(1, '../')
from config import weather_api_key
from faker import Faker
from citipy import citipy
import json
import inspect, re

fake = Faker()
Faker.seed(0)

In [20]:
# Create 2,000 random lat longs and get the nearest city.
cities = []
for _ in range(2000):
    city = {}
    latlng = fake.latlng()
    nearest_city = citipy.nearest_city(float(latlng[0]), float(latlng[1]))
    city['city'] = nearest_city.city_name
    city['county'] = nearest_city.country_code
    cities.append(city)
cities = pd.DataFrame(cities)
cities

,city,county
0,ushuaia,ar
1,urdzhar,kz
2,nouakchott,mr
3,marsala,it
4,pangnirtung,ca
...,...,...
1995,ushuaia,ar
1996,punta arenas,cl
1997,amderma,ru
1998,yellowknife,ca


In [28]:
# Build the urls for the api call.
target_url = "https://api.openweathermap.org/data/2.5/weather?appid=" + weather_api_key + "&units=imperial"
cities['search_query'] = cities['city'].replace(" ", "+", regex=True)
cities['url'] = target_url + "&q=" + cities['search_query']
pd.options.display.max_colwidth = 200
cities

,city,county,search_query,url
0,ushuaia,ar,ushuaia,https://api.openweathermap.org/data/2.5/weather?appid=d576c2a3f77df113d247c8daeb1b9bc3&units=imperial&q=ushuaia
1,urdzhar,kz,urdzhar,https://api.openweathermap.org/data/2.5/weather?appid=d576c2a3f77df113d247c8daeb1b9bc3&units=imperial&q=urdzhar
2,nouakchott,mr,nouakchott,https://api.openweathermap.org/data/2.5/weather?appid=d576c2a3f77df113d247c8daeb1b9bc3&units=imperial&q=nouakchott
3,marsala,it,marsala,https://api.openweathermap.org/data/2.5/weather?appid=d576c2a3f77df113d247c8daeb1b9bc3&units=imperial&q=marsala
4,pangnirtung,ca,pangnirtung,https://api.openweathermap.org/data/2.5/weather?appid=d576c2a3f77df113d247c8daeb1b9bc3&units=imperial&q=pangnirtung
...,...,...,...,...
1995,ushuaia,ar,ushuaia,https://api.openweathermap.org/data/2.5/weather?appid=d576c2a3f77df113d247c8daeb1b9bc3&units=imperial&q=ushuaia
1996,punta arenas,cl,punta+arenas,https://api.openweathermap.org/data/2.5/weather?appid=d576c2a3f77df113d247c8daeb1b9bc3&units=imperial&q=punta+arenas
1997,amderma,ru,amderma,https://api.openweathermap.org/data/2.5/weather?appid=d576c2a3f77df113d247c8daeb1b9bc3&units=imperial&q=amderma
1998,yellowknife,ca,yellowknife,https://api.openweathermap.org/data/2.5/weather?appid=d576c2a3f77df113d247c8daeb1b9bc3&units=imperial&q=yellowknife


In [29]:
# Make api call and retrieve data
city_data = []
for city in cities.index:
    try:
        new_city = {}
        response = requests.get(cities['url'][city]).json()
        new_city['name'] = cities['city'][city]
        new_city['country'] = cities['county'][city]
        new_city['lat'] = response['coord']['lat']
        new_city['lng'] = response['coord']['lon']
        new_city['max_temp'] = response["main"]["temp_max"]
        new_city['percent_humidity'] = response['main']['humidity']
        new_city['percent_cloudiness'] = response['clouds']['all']
        new_city['wind_speed'] = response['wind']['speed']
        new_city['current_description'] = response['weather'][0]['description']
        city_data.append(new_city)
    except:
        print('City not found. Skipping...')
        pass
city_data = pd.DataFrame(city_data)
city_data

City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skip

,name,country,lat,lng,max_temp,percent_humidity,percent_cloudiness,wind_speed,current_description
0,ushuaia,ar,-54.8000,-68.3000,51.80,87,75,5.75,broken clouds
1,nouakchott,mr,18.0858,-15.9785,59.00,33,0,9.22,clear sky
2,marsala,it,37.7966,12.4352,59.00,82,40,14.97,scattered clouds
3,pangnirtung,ca,66.1451,-65.7125,21.15,77,0,7.09,clear sky
4,miles city,us,46.4083,-105.8406,-7.60,64,75,5.59,broken clouds
...,...,...,...,...,...,...,...,...,...
1820,victoria,sc,22.2855,114.1577,77.00,77,18,5.01,few clouds
1821,ushuaia,ar,-54.8000,-68.3000,51.80,87,75,5.75,broken clouds
1822,punta arenas,cl,-53.1500,-70.9167,60.80,44,75,12.66,broken clouds
1823,yellowknife,ca,62.4560,-114.3525,-33.00,67,40,8.05,scattered clouds


In [30]:
# Write the files to csv
filepath = os.path.join("WeatherPy_Database.csv")
city_data.to_csv(filepath, index=False)